<a href="https://colab.research.google.com/github/izzul112/ML_MLO-Submission_1/blob/main/mlop_submission1_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Submission 1: Klasifikasi Tweet di Twitter**

Nama: Habib Azizul Haq

Username dicoding: kuroba_izzul

Dizaman sekarang kita sering menghabiskan waktu di media sosial, baik sekedar melihat-lihat saja atau memantau idola kita di media sosialnya. Tak jarang saat kita sedang berselancar di media sosial kita menemukan tweet yang dirasa kurang sedap atau terlalu vulgar. Tentu kita akan merasa risih dan terganggu oleh hal tersebut.

Untuk mengatasi masalah tersebut kita bisa membuat sebuah model machine learning untuk mengklasifikasikan sebuah tweet, apakah tweet itu bermuatan posotif atau justru negatif.

## **Tahap Persiapan**

Di tahap ini kita akan menginsatall semua library yang di butuhkan serta membuat virtual environment.

1. Install condacolab


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

^C
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...


KeyboardInterrupt: ignored

2. Cek versinya jika perlu

In [ ]:
!conda --version

3. Buat conda virtual environment, kita berinama mlops-tfx-sub-1-tweet

In [ ]:
!conda create --name mlops-tfx-sub-1-tweet python==3.9.15

4. Aktifkan conda virtual environment-nya

In [ ]:
!conda activate mlops-tfx-sub-1-tweet

5. Kita clone repository github ML_MLO-Submission_1, di dalamnya terdapat dataset yang akan kita kunakan nanti dan requirements yang akan kita gunakan.

In [ ]:
!git clone https://github.com/izzul112/ML_MLO-Submission_1.git

6. Selanjutnya kita install semua library yang dibutuhkan untuk menjalankan proyek ini. Dengan menjalankan perintah berikut.

In [ ]:
!pip install jupyter scikit-learn tensorflow tfx==1.11.0 flask joblib

7. Setelah itu kita install juga requirements yang di butuhkan.

In [ ]:
!pip install -r /content/ML_MLO-Submission_1/requirements.txt --user

8. Terakhir kita import Tensorflow dan component dari TensorFlow Extended (TFX).

In [ ]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## **Set Variabel**

Disini kita akan mendefinisikan variabel yang akan kita gunakan nanti.

In [ ]:
PIPELINE_NAME = "emailspam-pipeline"
SCHEMA_PIPELINE_NAME = "emailspam-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

Pastikan lokasi url dari variabel DATA_ROOT sudah benar dan didalamnya sudah ada datasetnya.

In [ ]:
DATA_ROOT = '/content/ML_MLO-Submission_1/data'

## **Membuat Tahapan Data Ingestion**

Dalam tahap ini kita akan melakukan proses pengumpulan data dari berbagai sumber dan memastikan data tersebut memiliki format yang dapat diproses oleh machine learning pipeline. 

1. Kita bagi terlebih dahulu dataset kita menjadi 'train' untuk proses training dan 'eval' untuk proses evaluasi dengan perbandingan 8:2.

In [ ]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

2. setelah itu kita memanggil fungsi InteractiveContext() untuk menampilkan komponen TFX secara iteraktif, kita definisikan variabelnya interactive_context untuk memanggil fungsi InteractiveContext() dengan parameter pipeline_root yang kita isi juga dengan variabel PIPELINE_ROOT yang sudah kita definisikan di bagian Set Vaiabel.

In [ ]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

3. Setelah kita definisikan kita jalankan untuk melihat tampilan dari example_gen dengan kode dibawah.

In [ ]:
interactive_context.run(example_gen)

## **Membuat Tahapan Data Validation**

Dibagian ini kita akan melihat dan menganal dataset kita lebih dalam. Proses ini memiliki 3 tahap diantaranya:
1. Membuat summary statistic
2. Membuat data schema
3. Mengidentifikasi anomali pada dataset

**1. Membuat summary statistic**

Kita akan membuat summary statistic menggunakan komponen **StatisticsGen()**. Komponen memiliki parameter input berupa examples untuk menerima dataset dari komponen ExampleGen.

Setelah itu kita jalankan untuk melihat tampilan dari statistics_gen dengan kode `interactive_context.run(statistics_gen)`.

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

Untuk tampilan lebih menarik kita akan menggunakan **interactive_context.show()** untuk menampilkan summary statistic yang telah dibuat.

In [ ]:
interactive_context.show(statistics_gen.outputs["statistics"])

**2. Membuat data schema**

Setelah membuat summary statistic, kita akan membuat data schema. Kita menggunakan komponen **ShcemaGen()** untuk membuat data schema. Komponen ini akan menerima input berupa summary statistic.

Setelah itu kita jalankan untuk melihat tampilan dari schema_gen dengan kode `interactive_context.run(schema_gen)`.

In [ ]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

Untuk tampilan lebih menarik kita akan menggunakan **interactive_context.show()** untuk menampilkan schema_gen yang telah dibuat.

In [ ]:
interactive_context.show(schema_gen.outputs["schema"])

Dapat dilihat dataset kita diatas memiliki 3 fitur yaitu "id", "label" dan "tweet", tapi kita hanya akan memakai 2 fitur saja yaitu "label" dan "tweet", karena fitur "id" hanya berisi nomor urut. Selain itu, berdasarkan data schema di atas ketiga fitur tersebut haruslah lengkap untuk setiap dataset.

**3. Mengidentifikasi anomali pada dataset**

Tahap selanjutnya adalah mengidentifikasi anomali pada dataset. Proses ini dilakukan menggunakan komponen **ExampleValidator()** yang telah disediakan oleh TFX. Komponen ini membutuhkan keluaran dari **statistics_gen** dan **schema_gen** sebagai parameter input.

Setelah itu kita jalankan untuk melihat tampilan dari example_validator dengan kode `interactive_context.run(example_validator)`.

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

Untuk tampilan lebih menarik kita akan menggunakan **interactive_context.show()** untuk menampilkan example_validator yang telah dibuat.

In [ ]:
interactive_context.show(example_validator.outputs['anomalies'])

Terlihat dari hasil diatas, dataset kita tidak memiliki aniomali.

## **Membuat Tahapan Data Preprocessing**

Setelah melakukan data validation, tahap berikutnya adalah melakukan data preprocessing. Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model. Pada latihan ini, kita menggunakan TFT dan komponen Transform untuk melakukan data preprocessing.

Ketika melakukan data preprocessing, Anda sangat disarankan untuk menempatkan preprocessing function ke dalam sebuah module tersendiri. Hal ini dilakukan karena komponen Transform akan menerima inputan berupa module file yang berisi preprocessing function. 

Sebelum membuat module, kita perlu mendefinisikan nama dari module tersebut.

In [ ]:
TRANSFORM_MODULE_FILE = "tweet_transform.py"

Selanjutnya, kita dapat membuat sebuah module dengan kode berikut.

Module ini berisi dua buah function, yaitu `transformed_name()` dan `preprocessing_fn()`. 

1. Function `transformed_name()` digunakan untuk mengubah nama fitur yang telah di-transform. Perubahan ini dilakukan dengan menambahkan “_xf” pada nama fitur tersebut.

2. function `preprocessing_fn()`, kita hanya menerapkan tahapan preprocessing yang sederhana. Tahapan yang dilakukan hanya mengubah data yang terdapat dalam fitur "tweet" ke dalam bentuk lowercase. Selain itu, kita juga memastikan data pada fitur "label" memiliki format integer (tf.int64).

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
LABEL_KEY = "label"
FEATURE_KEY = "tweet"
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Kode di atas akan menghasilkan sebuah module dengan nama tweet_transform.py.

Tahap selanjutnya adalah mendefinisikan komponen **Transform()**. Komponen ini akan menerima input berupa **examples** untuk menerima dataset dari komponen **ExampleGen**, **schema** untuk menerima data schema dari **SchemaGen**, dan **module_file** untuk menerima module file yang berisi **preprocessing function**.

Setelah itu kita jalankan untuk melihat tampilan dari transform dengan kode `interactive_context.run(transform)`.

In [ ]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Berdasarkan gambar di atas, dapat dilihat bahwa komponen **Transform** akan menghasilkan beberapa output mulai dari transform_graph, transformed_examples (transformed data), dll. Keseluruhan output nantinya digunakan untuk melakukan preprocessing terhadap serving set. Hal ini akan membantu kita untuk memastikan preprocessing yang dilakukan pada fase training sama dengan fase deployment sehingga dapat mencegah feature skew.

## **Membuat Tahapan Pengembangan Model**

Pada tahap ini, kita akan membuat dan melatih model menggunakan komponen Trainer. Komponen Trainer akan membutuhkan beberapa inputan salah satunya adalah sebuah module file yang berisi *training function*. Pada latihan ini, kita akan membuat sebuah module file berisi beberapa fungsi berikut.

**1. Fungsi transformed_name()**

Fungsi ini digunakan untuk mengubah nama fitur yang telah melalui proses transform.

In [ ]:
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

**2. Fungsi gzip_reader_fn()**

Fungsi ini digunakan untuk memuat data dalam format TFRecord.

In [ ]:
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

**3. Fungsi input_fn()**

Fungsi ini digunakan untuk memuat transformed_feature yang dihasilkan oleh komponen **Transform** dan membaginya ke dalam beberapa batch.

In [ ]:
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

**4. Fungsi model_builder()**

Fungsi ini bertanggung jawab dalam membuat arsitektur model. Kita menggunakan salah satu *embedding layer* yang tersedia dan dapat diunduh melalui [TensorFlow Hub](https://tfhub.dev/google/universal-sentence-encoder/4).

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 

**5. Fungsi _get_serve_tf_examples_fn()**

Fungsi ini digunakan untuk menjalankan tahapan preprocessing data pada *raw request data*.

In [ ]:
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn

**6. Fungsi run_fn()**

Fungsi ini bertanggung jawab untuk menjalankan proses training model sesuai dengan parameter training yang diberikan.

In [ ]:
from tfx.components.trainer.fn_args_utils import FnArgs

def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Nah, seluruh fungsi di atas, akan disatukan ke dalam sebuah module file. Namun, sebelum membuat module file, kita perlu mendefinisikan nama dari module tersebut.

In [ ]:
TRAINER_MODULE_FILE = "tweet_trainer.py"

Jika sudah buat semua fungsi yang sudah kita tulis di atas ke dalam sebuah file modul seperi kode di bawah.

In [ ]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "label"
FEATURE_KEY = "tweet"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Kode di atas akan menghasilkan sebuah module dengan nama tweet_trainer.py. Module tersebut mengandung training function beserta beberapa helper function.

Tahap selanjutnya adalah mendefinisikan komponen **Trainer()**. Komponen ini akan menerima beberapa input seperti berikut. 

* **module_file** untuk menerima module file yang mengandung mengandung training 
function beserta beberapa helper function.
* **examples** untuk menerima dataset dari komponen ExampleGen.
* **schema** untuk menerima data schema dari komponen SchemaGen.
* **tansform_graph** untuk menerima transform graph yang dihasilkan dari komponen Transform.
* **train_args** untuk menampung parameter training.
* **eval_args** untuk menampung parameter testing atau evaluation.

Setelah itu kita jalankan untuk melihat tampilan dari trainer dengan kode `interactive_context.run(trainer)`.

In [ ]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

## **Membuat Tahapan Analisis dan Validasi Model**

Apabila proses pengembangan model berjalan dengan lancar, tahap selanjutnya adalah membuat tahapan analisis dan validasi model. Untuk menjalankan tahapan ini kita perlu mendefinisikan dua buah komponen seperti berikut.

1. **Komponen Resolver**
2. **Komponen Evaluator**

**1. Komponen Resolver**

Untuk melakukan analisis dan validasi model, kita perlu menyediakan sebuah baseline model. Hal ini sangat penting terutama ketika kita memiliki lebih dari satu versi model dan ingin membandingkan dua buah versi model yang berbeda. Untuk melakukannya, kita bisa memanfaatkan komponen **Resolver()**.

Setelah itu kita jalankan untuk melihat tampilan dari model_resolver dengan kode `interactive_context.run(model_resolver)`.

In [ ]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

**2. Komponen Evaluator**

Setelah mendefinisikan komponen Resolver, tahap selanjutnya adalah membuat beberapa konfigurasi untuk mengevaluasi model. 

Kita akan mengatur beberapa konfigurasi seperti metrik untuk mengevaluasi model beserta nilai threshold dari suatu metrik. Selain menentukan metrik, kita juga dapat mengatur pembagian kelompok data berdasarkan fitur tertentu pada parameter slicing_specs

Konfigurasi ini dibuat menggunakan library TFMA seperti berikut.

In [ ]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

Setelah membuat konfigurasi yang akan digunakan untuk mengevaluasi model, tahap berikutnya adalah mendefinisikan komponen Evaluator. Komponen ini akan menerima beberapa input seperti berikut.

* **examples** untuk menerima dataset dari komponen ExampleGen.
* **model** untuk menerima model yang dihasilkan dari komponen Trainer.
* **baseline_model** untuk menampung baseline model yang disediakan oleh komponen Resolver.
* **eval_config** untuk menerima konfigurasi untuk mengevaluasi model.

Setelah itu kita jalankan untuk melihat tampilan dari evaluator dengan kode `interactive_context.run(evaluator)`.

In [ ]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Hasil evaluasi dari komponen evaluator dapat kita visualisasikan menggunakan library TFMA seperti contoh kode di bawah ini.

In [ ]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

## **Menambahkan Komponen Pusher**

In [ ]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/sarchasm-detection-model'))
 
)
 
interactive_context.run(pusher)